# Track Cluster

In [23]:
import pandas as pd
import numpy as np
from utils import utils
from utils import myplot
df = pd.read_csv(r'wifi_track_data\dacang\track_data\dacang_track_data_3_final_1220.csv')
df.t = pd.to_datetime(df.t)
mac_list = df.m.unique()
df_wifipos = pd.read_csv(r'wifi_track_data\dacang\pos_data\wifi_pos_new_3_1220.csv')
from tqdm import tqdm
from importlib import reload

import warnings
warnings.filterwarnings('ignore')

## 初步清理

In [24]:
#清理只经过2个及以下wifi的mac
del_mac = []
for m in mac_list:
    df_now = utils.GetDfNow(df,m)
    if len(df_now.a.unique())<=2:
        del_mac.append(m)
df = df[~df.m.isin(del_mac)]
mac_list = df.m.unique()

In [25]:
mac_list = df['m'].unique()
ori_mac_list = df.oriMac.unique()
ori_Mac = []
data_count = []
Dur = []
wifi_count = []
active_moment = []
middle_moment = []
moving_dis = []
for m in mac_list:
    ori_Mac.append(m.split('-')[1])
    df_now = utils.GetDfNow(df, m)
    data_count.append(len(df_now))
    t1 = df_now.iloc[0].t
    t2 = df_now.iloc[len(df_now)-1].t
    t1 = t1.hour + t1.minute/60 + t1.second/3600
    t2 = t2.hour + t2.minute/60 + t2.second/3600
    Dur.append(t2-t1)
    middle_moment.append((t2+t1)/2)
    wifi_count.append(len(df_now['a'].unique()))
    h = 0
    row_last = df_now.iloc[0]
    dis = 0
    for i,row in df_now.iterrows():
        h += row.t.hour + row.t.minute/60 + row.t.second/3600
        if row.a != row_last.a:
            dis += utils.GetWifiTrackDistance(row.a,row_last.a,df_wifipos)
        row_last = row
    active_moment.append(h/len(df_now))
    moving_dis.append(dis)

df_sta = pd.DataFrame({'mac':mac_list,
                       'ori_mac':ori_Mac,
                       'data_count':data_count,
                       'Dur':Dur,
                       'wifi_count':wifi_count,
                       'active_moment':active_moment,
                       'moving_dis':moving_dis,
                       'middle_moment':middle_moment})
df_sta

,mac,ori_mac,data_count,Dur,wifi_count,active_moment,moving_dis,middle_moment
0,"7.5-48,74,38,133,29,213","48,74,38,133,29,213",5,7.137500,3,21.925500,255.84,20.428194
1,"7.5-48,74,38,155,214,98","48,74,38,155,214,98",9,7.140278,5,22.534414,437.00,20.429583
2,"7.6-48,74,38,155,214,98","48,74,38,155,214,98",44,23.999167,14,4.545069,1053.48,12.000139
3,"7.6-32,50,51,210,96,61","32,50,51,210,96,61",11,23.996389,4,10.667727,161.62,11.998750
4,"7.6-48,74,38,133,29,213","48,74,38,133,29,213",15,23.996389,6,9.514241,582.27,11.998750
...,...,...,...,...,...,...,...,...
1308,"8.7-252,4,28,35,119,153","252,4,28,35,119,153",14,16.207500,4,15.388036,2014.83,15.774306
1309,"8.7-124,161,119,201,50,40","124,161,119,201,50,40",32,13.351667,6,16.939488,2149.56,17.204444
1310,"8.7-56,251,20,50,26,70","56,251,20,50,26,70",6,8.996944,3,18.492269,363.15,15.832361
1311,"8.7-144,173,247,200,14,93","144,173,247,200,14,93",10,8.199167,4,18.670944,559.52,15.882361


In [26]:
myplot.Boxes([(df_sta.data_count,"数据量"),
              (df_sta.Dur,"持续时间"),
              (df_sta.wifi_count,"wifi数量"),
              (df_sta.active_moment,"活跃时刻"),
              (df_sta.moving_dis,"移动距离"),
              (df_sta.middle_moment,"中间时刻")])

In [29]:
#清理中间时刻在凌晨或深夜的mac
df_sta = df_sta[(df_sta.middle_moment>6) & (df_sta.middle_moment<20)]

## 清除离群值
随机孤立森林

In [31]:
#归一化
df_outlier = df_sta[['data_count','Dur','active_moment','moving_dis','wifi_count','middle_moment']]
df_outlier = utils.Normalize_df(df_outlier,['data_count','Dur','active_moment','moving_dis','wifi_count'],mul_index=10)
df_outlier

,data_count,Dur,active_moment,moving_dis,wifi_count,middle_moment
2,2.836879,9.999884,1.332136,1.132853,4.4,12.000139
3,0.496454,9.998726,4.344526,0.147217,0.4,11.998750
4,0.780142,9.998726,3.777003,0.612097,1.2,11.998750
5,0.141844,9.997684,4.595270,0.092821,0.0,11.997778
6,0.425532,9.994905,3.448401,0.212089,0.0,12.000556
...,...,...,...,...,...,...
1308,0.709220,6.752003,6.666951,2.195286,0.4,15.774306
1309,1.985816,5.561577,7.430276,2.344183,1.2,17.204444
1310,0.141844,3.746353,8.194255,0.369937,0.0,15.832361
1311,0.425532,3.413807,8.282165,0.586955,0.4,15.882361


In [34]:
from sklearn.ensemble import IsolationForest
rng = np.random.RandomState(42)
model_IF = IsolationForest(max_samples=2000,random_state=rng,contamination='auto')
model_IF.fit(df_outlier[['data_count','Dur','active_moment','moving_dis','wifi_count','middle_moment']])
df_outlier['label'] = model_IF.predict(df_outlier[['data_count','Dur','active_moment','moving_dis','wifi_count','middle_moment']])
print(df_outlier.label.value_counts())

myplot.Scatter_Matrix(df_outlier,
                      dimentions = ['data_count',
                                    'Dur',
                                    'active_moment',
                                    'moving_dis',
                                    'middle_moment',
                                    ],
                        color = 'label')

label
 1    1088
-1     144
Name: count, dtype: int64


In [35]:
df_cluster = df_sta[['mac','ori_mac','data_count','Dur','active_moment','moving_dis','wifi_count','middle_moment']][df_outlier.label == 1].reset_index(drop=True)
df_cluster = utils.Normalize_df(df_cluster,['data_count','Dur','active_moment','moving_dis','wifi_count','middle_moment'],mul_index=10)

df_cluster

,mac,ori_mac,data_count,Dur,active_moment,moving_dis,wifi_count,middle_moment
0,"7.6-48,74,38,155,214,98","48,74,38,155,214,98",4.431818,10.000000,0.583474,2.375316,9.166667,4.456263
1,"7.6-32,50,51,210,96,61","32,50,51,210,96,61",0.681818,9.998841,4.098094,0.308677,0.833333,4.455205
2,"7.6-48,74,38,133,29,213","48,74,38,133,29,213",1.136364,9.998841,3.435952,1.283417,2.500000,4.455205
3,"7.6-20,178,229,137,47,142","20,178,229,137,47,142",0.113636,9.997797,4.390642,0.194624,0.000000,4.454464
4,"7.6-4,207,140,11,128,186","4,207,140,11,128,186",0.568182,9.995015,3.052566,0.444698,0.000000,4.456580
...,...,...,...,...,...,...,...,...
1083,"8.7-76,2,32,111,66,59","76,2,32,111,66,59",0.113636,4.137164,5.812362,0.828685,0.833333,4.915807
1084,"8.7-252,4,28,35,119,153","252,4,28,35,119,153",1.022727,6.748279,6.807716,4.602979,0.833333,7.332275
1085,"8.7-124,161,119,201,50,40","124,161,119,201,50,40",3.068182,5.556444,7.698304,4.915178,2.500000,8.422078
1086,"8.7-56,251,20,50,26,70","56,251,20,50,26,70",0.113636,3.739074,8.589655,0.775667,0.000000,7.376515


In [36]:
myplot.Scatter_Matrix(df_cluster,
                      dimentions = ['data_count',
                                    'Dur',
                                    'active_moment',
                                    'moving_dis',
                                    'middle_moment',
                                    'wifi_count'])

## 简单的居民、游客、路过人群分类
- 当一天内待在区域超过10小时的天数超过10天时，被视为居民
- 当一天内待在区域小于0.5小时的天数超过5天时，被视为路过
- 其余视为游客

In [37]:
def DivideId(df_now):
    days = df_now.calender.unique()
    count = 0
    isPassing = 0
    for i in range(len(days)):
        df_one = df_now[df_now.calender == days[i]]
        time = df_one.iloc[len(df_one)-1].t - df_one.iloc[0].t
        h = time.components.hours
        if h>5:
            count += 1
        elif h<0.5:
            isPassing += 1
    if isPassing > 5:
        return 'pass'
    elif count < 4:
        return 'tourist'
    else:
        return 'resident'
    

tourist_list = []
resident_list = []
pass_list = []

for mac in tqdm(ori_mac_list,desc="Dividing:"):
    df_now = df[df.oriMac == mac]
    df_now.sort_values(by='t')
    result = DivideId(df_now)
    if result == 'pass':
        pass_list.append(mac)
    elif result == 'tourist':
        tourist_list.append(mac)
    else:
        resident_list.append(mac)

# add id to df_sta
def GetId(m):
    if m in tourist_list:
        return 1
    elif m in resident_list:
        return 0
    else:
        return -1
df_sta['id'] = df_sta.ori_mac.apply(GetId)
df_cluster['id'] = df_sta.ori_mac.apply(GetId)

print('pass count:',len(pass_list))
print('tourists count:',len(tourist_list))
print('residents count:',len(resident_list))


Dividing:: 100%|██████████| 312/312 [00:00<00:00, 391.64it/s]

pass count: 0
tourists count: 233
residents count: 79


In [38]:
myplot.Scatter_Matrix(df_cluster,
                      dimentions = ['data_count',
                                    'Dur',
                                    'active_moment',
                                    'moving_dis',
                                    'middle_moment',
                                    ],
                        color = 'id')

## K-means++聚类

In [39]:
#获得居民最佳聚类数
df_resident = df_cluster[df_cluster.id == 0]
df_score_resi = utils.GetKmeansClusterNumDf(df_resident,['Dur','data_count','active_moment','moving_dis','middle_moment'])
myplot.Double_Axes_Line(df_score_resi,"n_clusters","calinski","silhouette","聚类数","卡林斯基-哈拉巴斯指数","轮廓系数")

In [40]:
#获得游客最佳聚类数
df_tourist = df_cluster[df_cluster.id == 1]
df_score_tour = utils.GetKmeansClusterNumDf(df_tourist,['Dur','data_count','active_moment','moving_dis','middle_moment'])
myplot.Double_Axes_Line(df_score_tour,"n_clusters","calinski","silhouette","聚类数","卡林斯基-哈拉巴斯指数","轮廓系数")

In [41]:
df_resident['cluster'] = utils.Kmeans(df_resident,['Dur','data_count','active_moment','moving_dis','middle_moment'],4)
myplot.Scatter_Matrix(df_resident,
                      dimentions = ['data_count',
                                    'Dur',
                                    'active_moment',
                                    'moving_dis',
                                    'middle_moment',
                                    ],
                        color = 'cluster',
                        color_scale='Jet')

In [43]:
df_tourist['cluster'] = utils.Kmeans(df_tourist,['Dur','data_count','active_moment','moving_dis','middle_moment'],2)
myplot.Scatter_Matrix(df_tourist,
                      dimentions = ['data_count',
                                    'Dur',
                                    'active_moment',
                                    'moving_dis',
                                    'middle_moment',
                                    ],
                        color = 'cluster',
                        color_scale='Jet')

In [44]:
df_resident.to_csv(r"wifi_track_data\dacang\cluster_data\dacang_track_data_resident.csv",index=False)
df_tourist.to_csv(r"wifi_track_data\dacang\cluster_data\dacang_track_data_tourist.csv",index=False)